In [1]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd
import pycountry
from geojson import FeatureCollection, dump

In [2]:
load_dotenv()
TOKEN=os.getenv("MAPBOX_TOKEN")
df = pd.read_csv("../output/processed/clean.csv")
origins = list(df.origin.unique())
destinations = list(df.destination.unique())

In [3]:
in_first = set(origins)
in_second = set(destinations)
in_second_but_not_in_first = in_second - in_first
countries = origins + list(in_second_but_not_in_first)

In [4]:
df = pd.DataFrame({"country":countries})

In [ ]:
headers = {'Accept': 'application/json'}
lst = []
for i in countries:
    obj = {}
    obj["type"]="Feature"
    obj["properties"]={"name": pycountry.countries.get(alpha_2=i).name}
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{i}.json?&types=country&access_token={TOKEN}"
    r = requests.get(url)
    jason = r.json()
    obj["properties"]["code"] = i.lower()
    coords = jason["features"][0]["center"]
    obj["geometry"] = {"type":"Point", "coordinates": coords}
    lst.append(obj)
    df.loc[df["country"] == i, "lat"]=coords[0]
    df.loc[df["country"] == i, "lon"]=coords[1]

In [ ]:
with open('../output/processed/countries.geojson', 'w') as f:
    dump(FeatureCollection(lst), f)

In [ ]:
df

In [ ]:
df.to_csv("../output/processed/country_coords.csv", index=False)